In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm


In [ ]:
tomtom = pd.read_parquet("tomtom_data.parquet")
vessels = pd.read_parquet("vessels_data.parquet")
sensors_location = pd.read_csv("sensor-location.xlsx - Sheet1.csv", )
sensors = pd.read_csv("sensordata_SAIL2025.csv", parse_dates=["timestamp"])

sensors_location["Effectieve breedte"] = (
    sensors_location["Effectieve breedte"]
    .astype(str)
    .str.replace(",", ".")
    .astype(float)
)

In [22]:
vessels.info()
vessels.head()
# sensors.info
# sensors.head()
# sensors_location.head()
# sensors_location.info()
vessels["timestamp"].value_counts().sort_index().head(20)




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1059291 entries, 0 to 1059290
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   timestamp   1059291 non-null  object 
 1   imo-number  369211 non-null   object 
 2   length      1059150 non-null  float64
 3   lat         1059291 non-null  float64
 4   lon         1059291 non-null  float64
dtypes: float64(3), object(2)
memory usage: 40.4+ MB


timestamp
2025-08-20T06:28:39.232723Z    1
2025-08-20T06:28:41.419140Z    1
2025-08-20T06:28:43.647668Z    1
2025-08-20T06:29:08.685948Z    1
2025-08-20T06:29:10.337458Z    1
2025-08-20T06:29:10.469965Z    1
2025-08-20T06:29:11.533956Z    1
2025-08-20T06:29:15.947110Z    1
2025-08-20T06:29:37.965244Z    1
2025-08-20T06:29:40.662814Z    1
2025-08-20T06:29:44.178851Z    1
2025-08-20T06:29:44.332312Z    1
2025-08-20T06:30:08.173798Z    1
2025-08-20T06:30:11.924080Z    1
2025-08-20T06:30:14.923881Z    1
2025-08-20T06:30:37.940285Z    1
2025-08-20T06:30:40.859028Z    1
2025-08-20T06:30:43.095060Z    1
2025-08-20T06:30:44.524852Z    1
2025-08-20T06:31:08.081904Z    1
Name: count, dtype: int64

In [ ]:
#Find Flow per sensor per meter
#Divide by 3 minutes and effective width

width = sensors_location.set_index("Objectummer")["Effectieve breedte"].to_dict()

# Divide each matching visitor column by its sensor’s width
for col in sensors.columns:
    if "_" in col:  # e.g. CMSA-GAKH-01_0
        sensor_id = col.split("_")[0]
        if sensor_id in width:
            sensors[col] = sensors[col] / (3*width[sensor_id])


In [ ]:
#Combine vessel data into 3 min bins
vessels["timestamp"] = pd.to_datetime(vessels["timestamp"], utc=True, errors="coerce")
vessels["timestamp"] = vessels["timestamp"].dt.floor("3min")

vessels = (
    vessels.groupby(["timestamp", "imo-number"], as_index=False)
    .agg({
        "lat": "mean",
        "lon": "mean",
        "length": "first"
    })
)

vessels.info()
vessels.head(100


)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41844 entries, 0 to 41843
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   timestamp   41844 non-null  datetime64[ns, UTC]
 1   imo-number  41844 non-null  object             
 2   lat         41844 non-null  float64            
 3   lon         41844 non-null  float64            
 4   length      41844 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(3), object(1)
memory usage: 1.6+ MB


,timestamp,imo-number,lat,lon,length
0,2025-08-20 06:27:00+00:00,4becccd38142efcac823c94e99e4c9f918dc47de027d83...,52.059721,3.616344,14000.0
1,2025-08-20 06:27:00+00:00,64a15227c0878ef130cc4e6c27fdc668b69e528a557b63...,53.661851,5.537259,19900.0
2,2025-08-20 06:27:00+00:00,a867f56d1cd3b9b6812798549281aa82c60f5cebcf5f61...,52.086418,3.423325,19900.0
3,2025-08-20 06:27:00+00:00,b2eb0407183a1fabe106857e32added9ec78ac9c079149...,51.947955,4.052420,33600.0
4,2025-08-20 06:30:00+00:00,2d18f769b33844885ab5d48e9b4c0764308170355a4245...,53.410521,4.589400,8900.0
...,...,...,...,...,...
95,2025-08-20 06:33:00+00:00,508607cb85db6e98b5293f54b5002389f6e18c082303f0...,52.406696,4.816781,5900.0
96,2025-08-20 06:33:00+00:00,51642004ecb740656103b2b0c3d4ce286db2253a1d930d...,52.406776,4.882029,8800.0
97,2025-08-20 06:33:00+00:00,51de12cb55ef030d7e33cd76f8c5badcae3330621021a1...,52.100666,4.267667,5100.0
98,2025-08-20 06:33:00+00:00,52bddda6694714952ff0993167c638d56677f44f639d87...,52.405651,4.818190,4100.0
